In [2]:
import math
import numpy as np

# Constants
A0 = 333  # Administered activity (MBq)
T = 1  # Occupancy factor1
T1_2 = 109.8 / 60  # Radionuclide half-life in hours (109.8 minutes)
dr = 0.092  # Dose rate immediately after administration (µSv m²/MBq h)
P_controlled = 100  # Weekly dose limit for controlled areas (µSv)
P_uncontrolled = 20  # Weekly dose limit for uncontrolled areas (µSv)

# Function to get user input for variables
def get_inputs():
    Nw = int(input("Enter Number of patients per week: "))
    d = float(input("Enter Distance from source to barrier (m): "))
    tU = float(input("Enter Uptake time (hours): "))
    print(f"Input values - Nw: {Nw}, d: {d}, tU: {tU},")
    return Nw, d, tU



# Function to calculate the ***dose reduction factor*** over uptake time
def RtU(T1_2, tU):
    rtU_value = 1.443 * (T1_2 / tU) * (1 - math.exp(-0.693 * tU / T1_2))
    print(f"RtU: {rtU_value}")
    return rtU_value

#The total dose at a point d meters from the patient during the uptake time(tU) is
def DtU(dr, A0, tU, RtU, d):
    dose_uptake = dr * A0 * tU * (RtU / d**2)
    print(f"DtU: {dose_uptake}")
    return dose_uptake


# Function to calculate transmission factor for the uptake room
def B_uptake_room(P, d, T, Nw, A0, tU, RtU):
    return 10.9 * P * (d ** 2) / (T * Nw * A0 * tU * RtU)



# Main function to run the calculation for both controlled and uncontrolled areas
def calculate_transmission_factors():
    # Get user inputs
    Nw, d, tU = get_inputs()
    
    # Calculate the dose reduction factor RtU & RtI
    rtU = RtU(T1_2, tU)

    # Calculate the transmission factor B for uncontrolled and controlled areas
    B_uc = B_uptake_room(P_uncontrolled, d, T, Nw, A0, tU, rtU)
    B_c = B_uptake_room(P_controlled, d, T, Nw, A0, tU, rtU)

    
     # Output the results
    print()  # This adds a blank line before the following prints
    print("****UPTAKE ROOM****")
    print()
    print("*********B********")
    print(f"\nTransmission Factor (B) for the uptake room (Uncontrolled area): {B_uc:.4f} ")
    print(f"Transmission Factor (B) for the uptake room (Controlled area): {B_c:.4f} ")

    return {
        "Uptake Room Uncontrolled": B_uc,
        "Uptake Room Controlled": B_c,

    }

# Run the calculations and store the results
B_results = calculate_transmission_factors()

# X = Shileding Thickness
# Lead Parameters
lead_params = (1.543, -0.4408, 2.136)  # (alpha, beta, gamma)
# concrete Parameters
concrete_params = (0.1539, -0.1161, 2.0752)  # (alpha, beta, gamma)

def calculate_x(alpha, beta,  gamma, B):
    # The Monte Carlo transmission data have been fitted  to the Model By the following parameters by Archer et al. 
    # #visit: https://pmc.ncbi.nlm.nih.gov/articles/PMC3155576/
    numerator = (B ** (-gamma)) + (beta / alpha)
    denominator = 1 + (beta / alpha)
    x = (1 / (alpha * gamma)) * math.log(numerator / denominator)
    return x

# Calculate thickness for uncontrolled area
B_uc = B_results["Uptake Room Uncontrolled"]
lead_thickness_uc = calculate_x(*lead_params, B_uc)
concrete_thickness_uc = calculate_x(*concrete_params, B_uc)

# Calculate thickness for controlled area
B_c = B_results["Uptake Room Controlled"]
lead_thickness_c = calculate_x(*lead_params, B_c)
concrete_thickness_c = calculate_x(*concrete_params, B_c)

# Output the thickness results
print()
print("****UnControl****")
print(f"Shielding thickness for lead (Uncontrolled Area): {lead_thickness_uc / 0.1:.4f} mm")
print(f"Shielding thickness for concrete (Uncontrolled Area): {concrete_thickness_uc:.4f} cm")
print()
print("****Control****")
print(f"Shielding thickness for lead (Controlled Area): {lead_thickness_c / 0.1:.4f} mm")
print(f"Shielding thickness for concrete (Controlled Area): {concrete_thickness_c:.4f} cm")

Input values - Nw: 48, d: 0.2, tU: 1.0,
RtU: 0.8324541127798754

****UPTAKE ROOM****

*********B********

Transmission Factor (B) for the uptake room (Uncontrolled area): 0.0007 
Transmission Factor (B) for the uptake room (Controlled area): 0.0033 

****UnControl****
Shielding thickness for lead (Uncontrolled Area): 48.5278 mm
Shielding thickness for concrete (Uncontrolled Area): 52.0267 cm

****Control****
Shielding thickness for lead (Controlled Area): 38.0973 mm
Shielding thickness for concrete (Controlled Area): 41.5690 cm
